In [12]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [33.6844, 24.8607, 31.5497, 34.0151, 30.1798, 30.1575, 31.4187, 33.5651, 32.4927, 32.1877, 25.396, 34.1463, 27.7094, 29.3544, 32.0836, 32.9474, 34.1992, 35.1991, 35.8508, 35.9221, 35.3333, 25.1266],
	"longitude": [73.0479, 67.0011, 74.3436, 71.5249, 66.975, 71.5249, 73.0791, 73.0169, 74.5313, 74.1945, 68.3578, 73.2117, 68.8574, 71.6911, 72.6711, 73.7285, 72.0448, 72.4258, 71.7864, 74.3087, 75.5667, 62.3254],
	"start_date": "2020-01-01",
	"end_date": "2024-03-23",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "pressure_msl", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "is_day"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(5).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(10).ValuesAsNumpy()
hourly_is_day = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["is_day"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)
hourly_dataframe.to_csv("weather_data.csv")

Coordinates 33.70825958251953°N 72.8944091796875°E
Elevation 527.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  relative_humidity_2m  \
0     2020-01-01 00:00:00+00:00        8.679000             70.831078   
1     2020-01-01 01:00:00+00:00        9.679000             62.140259   
2     2020-01-01 02:00:00+00:00       10.379000             55.619858   
3     2020-01-01 03:00:00+00:00       10.429000             55.632954   
4     2020-01-01 04:00:00+00:00        9.979000             71.830368   
...                         ...             ...                   ...   
37051 2024-03-23 19:00:00+00:00       18.429001             59.626328   
37052 2024-03-23 20:00:00+00:00       17.329000             66.501236   
37053 2024-03-23 21:00:00+00:00       16.279001             71.792770   
37054 2024-03-23 22:00:00+00:00       15.929000             75.632240   
37055 2024-03-23 23:00:00+00:00       15.879000             77.390800   

In [14]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [28.6139, 19.076, 12.9716, 13.0827, 22.5726, 18.5204, 23.0225, 17.385, 26.9124, 21.1702, 25.3176, 22.3072, 15.2993, 24.5854, 21.1458, 11.0168, 22.7196, 19.9975, 10.8505, 15.8281],
	"longitude": [77.209, 72.8777, 77.5946, 80.2707, 88.3639, 73.8567, 72.5714, 78.4867, 75.7873, 72.8311, 82.9739, 73.1812, 74.124, 73.7125, 79.0882, 76.9558, 75.8577, 73.7898, 76.2711, 78.0373],
	"start_date": "2020-01-01",
	"end_date": "2024-03-23",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "pressure_msl", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "is_day"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(5).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(10).ValuesAsNumpy()
hourly_is_day = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["is_day"] = hourly_is_day

df2 = pd.DataFrame(data = hourly_data)
df2 = df2.reindex(columns=hourly_dataframe.columns)

print(df2)
df_combined = pd.concat([hourly_dataframe, df2], ignore_index=True)
output_file_path = 'combined_csv_file.csv'
df_combined.to_csv(output_file_path, index=False)



Coordinates 28.646747589111328°N 77.17218017578125°E
Elevation 214.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  relative_humidity_2m  \
0     2020-01-01 00:00:00+00:00        3.282500             98.596809   
1     2020-01-01 01:00:00+00:00        3.032500             98.593979   
2     2020-01-01 02:00:00+00:00        3.282500             98.945969   
3     2020-01-01 03:00:00+00:00        5.882500             89.151871   
4     2020-01-01 04:00:00+00:00        9.232500             77.129669   
...                         ...             ...                   ...   
37051 2024-03-23 19:00:00+00:00       22.832499             42.954018   
37052 2024-03-23 20:00:00+00:00       22.132500             45.730862   
37053 2024-03-23 21:00:00+00:00       21.282499             50.139198   
37054 2024-03-23 22:00:00+00:00       21.282499             50.642651   
37055 2024-03-23 23:00:00+00:00       19.632500             56.631882 

In [17]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [23.8103, 22.3475, 24.3636, 24.8949, 22.701, 27.7172, 28.2096, 27.4765, 26.6904, 27.591, 27.4728, 27.5142, 26.8767, 27.1585, 26.9039, 16.8409, 19.7633, 21.9162, 20.1543, 16.8661],
	"longitude": [90.4125, 91.8123, 88.6241, 91.8687, 90.3535, 85.324, 83.9856, 89.6362, 87.2733, 84.5, 89.6393, 90.4321, 91.4203, 90.7515, 89.3888, 96.1735, 96.0785, 95.9559, 94.9325, 96.1951],
	"start_date": "2020-01-01",
	"end_date": "2024-03-23",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "pressure_msl", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "is_day"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(5).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(10).ValuesAsNumpy()
hourly_is_day = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["is_day"] = hourly_is_day

df3 = pd.DataFrame(data = hourly_data)
df3 = df3.reindex(columns=df2.columns)
print(df3)
df_combined = pd.concat([df2, df3], ignore_index=True)
output_file_path = 'combined_csv_file.csv'
df_combined.to_csv(output_file_path, index=False)




Coordinates 23.796133041381836°N 90.38054656982422°E
Elevation 19.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  relative_humidity_2m  \
0     2020-01-01 00:00:00+00:00       15.298000             85.599785   
1     2020-01-01 01:00:00+00:00       15.848000             83.451019   
2     2020-01-01 02:00:00+00:00       17.298000             80.434181   
3     2020-01-01 03:00:00+00:00       19.898001             74.820107   
4     2020-01-01 04:00:00+00:00       21.997999             68.335304   
...                         ...             ...                   ...   
37051 2024-03-23 19:00:00+00:00       22.548000             86.590042   
37052 2024-03-23 20:00:00+00:00       22.598000             91.249290   
37053 2024-03-23 21:00:00+00:00       21.948000             93.196526   
37054 2024-03-23 22:00:00+00:00       22.497999             91.523697   
37055 2024-03-23 23:00:00+00:00       22.747999             92.951340  

In [18]:
import pandas as pd

# Read the first CSV file
file_path1 = 'combined_csv_file.csv'
df1 = pd.read_csv(file_path1)

# Read the second CSV file
file_path2 = 'combined_csv_file2.csv'
df2 = pd.read_csv(file_path2)

# Concatenate the DataFrames (appends the second file to the first)
df_combined = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a new CSV file
output_file_path = 'combined_csv_file3.csv'
df_combined.to_csv(output_file_path, index=False)

print(f"Files combined and saved to {output_file_path}.")


Files combined and saved to combined_csv_file3.csv.
